# Data Cleanup

## Import Libraries

In [1]:
import sys
print ('Python version: {}'.format(sys.version))
import pandas as pd
print ('pandas version: {}'.format(pd.__version__))
import matplotlib
print ('matplotlib version: {}'.format(matplotlib.__version__))
import numpy as np
print ('NumPy version: {}'.format(np.__version__))
import scipy as sp
print ('SciPy version: {}'.format(sp.__version__))
import IPython
print ('IPython version: {}'.format(IPython.__version__))
import sklearn
print ('scikit-learn version: {}'.format(sklearn.__version__))
import random
import time
import warnings
import copy
warnings.filterwarnings('ignore')
print ('-*'*20)
from subprocess import check_output
print (check_output(['ls', './data']).decode('utf8'))

Python version: 3.9.1 (v3.9.1:1e5d33e9b9, Dec  7 2020, 12:10:52) 
[Clang 6.0 (clang-600.0.57)]
pandas version: 1.2.3
matplotlib version: 3.3.3
NumPy version: 1.19.5
SciPy version: 1.6.0
IPython version: 7.21.0
scikit-learn version: 0.24.1
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
2019_Crash_1_Database.csv
CODE_TB.csv



## Import Data

We will bring in the data as data_originalk, and make a deep copy, data_raw.
Then we will process each column, copy it to data, and delete it from data_raw

In [2]:
data_original = pd.read_csv('./data/2019_Crash_1_Database.csv', parse_dates = ['crash_date', 'crash_hour', 'crash_time'])
data_raw = data_original.copy(deep=True)
data = pd.DataFrame()

# Dependent Variable
The dependent variable y=1 if somebody died, y=0 otherwise.  
We have two fields that give us this information.  
'num_tot_kil' [0, 1, 2, 3, 4], giving the number killed.

'severity_cd' ['E' 'D' 'C' 'B' 'A'], with 'A' being 'Fatal.'

Do these columns agree?  Yes (below).

In [3]:
for x in ['num_tot_kil', 'severity_cd']:
    print (data_raw[x].isnull().sum())
A = np.where( (
    (data_raw['num_tot_kil'] == 0) & (data_raw['severity_cd'] == 'A') |
    (data_raw['num_tot_kil'] > 0) & (data_raw['severity_cd'] != 'A') 
))
print (A)
#A = np.where( (data_raw['num_tot_kil']>0 & data_raw['severity_cd']=='A'))

0
0
(array([], dtype=int64),)


In [4]:
data['fatal'] = data_raw['num_tot_kil'].apply(lambda x: 1 if x>0 else x)
for x in ['num_tot_kil', 'severity_cd']:
    data_raw.drop([x], axis=1, inplace=True)

# No Idea What This Means

In [5]:
for x in ['quadrant', 'spotted_by', 'city_cd', 'bypass']:
    data_raw.drop([x], axis=1, inplace=True)

# Time

- crash_date
- crash_hour
- crash_year
- crash_time

The years are all the same.

What might be interesting is the month and the day of week.

In [6]:
for x in ['crash_date', 'crash_hour']:
    print (data_raw[x].isnull().sum())


data['crash_month'] = data_raw['crash_date'].dt.month
data['crash_dayofweek'] = data_raw['crash_date'].dt.dayofweek
data['crash_hour'] = data_raw['crash_hour']

data_raw.drop(['crash_date', 'crash_hour', 'crash_year', 'crash_time'], axis=1, inplace=True)


0
0


# Direction
I think I'll use just 'pri_road_dir,', which seems clean enough.  
Change the blanks to 'Z'.

In [7]:
for x in data_raw:
    if 'dir' in x:
        values = data_raw[x].unique()
        print (x, sorted(values, key=lambda x: (str(type(x)), x)))
data_raw['pri_road_dir'].replace([' '], 'Z', inplace=True)
data['pri_road_dir'] = data_raw['pri_road_dir']
data_raw.drop(['travel_dirs', 'pri_dir', 'pri_road_dir'], axis=1, inplace=True)

travel_dirs ['    ', 'E   ', 'EE  ', 'EEE ', 'EEEE', 'EEEN', 'EEES', 'EEEW', 'EEN ', 'EENE', 'EENN', 'EENW', 'EES ', 'EESS', 'EESW', 'EEW ', 'EEWE', 'EEWW', 'EN  ', 'ENE ', 'ENEE', 'ENN ', 'ENNE', 'ENNN', 'ENNS', 'ENS ', 'ENSS', 'ENSW', 'ENW ', 'ENWN', 'ENWW', 'ES  ', 'ESE ', 'ESEE', 'ESN ', 'ESNE', 'ESNN', 'ESS ', 'ESSE', 'ESSS', 'ESSW', 'ESW ', 'ESWW', 'EW  ', 'EWE ', 'EWEE', 'EWN ', 'EWNN', 'EWS ', 'EWSS', 'EWW ', 'EWWE', 'EWWS', 'EWWW', 'N   ', 'NE  ', 'NEE ', 'NEEE', 'NEEW', 'NEN ', 'NES ', 'NESN', 'NESS', 'NEW ', 'NEWE', 'NN  ', 'NNE ', 'NNEE', 'NNES', 'NNN ', 'NNNE', 'NNNN', 'NNNS', 'NNNW', 'NNS ', 'NNSN', 'NNSS', 'NNW ', 'NNWN', 'NNWW', 'NS  ', 'NSE ', 'NSEE', 'NSEN', 'NSN ', 'NSNN', 'NSNW', 'NSS ', 'NSSE', 'NSSS', 'NSW ', 'NSWW', 'NW  ', 'NWE ', 'NWEE', 'NWEN', 'NWN ', 'NWNS', 'NWS ', 'NWSS', 'NWW ', 'NWWW', 'S   ', 'S S ', 'SE  ', 'SEE ', 'SEEE', 'SEN ', 'SENN', 'SES ', 'SESS', 'SEW ', 'SEWW', 'SN  ', 'SNE ', 'SNEN', 'SNN ', 'SNNN', 'SNNS', 'SNS ', 'SNSN', 'SNSS', 'SNW ', 'SN

# Binary Features

In [8]:
for x in ['intersection', 'alcohol', 'roadway_departure', 'lane_departure', 'dr_sex_1', 'dr_sex_2']:
    data_raw[x].fillna(' ', inplace=True)
    data_raw[x].replace([' '], data_raw[x].mode(), inplace=True)
    data_raw[x].replace({'M':1, 'F':0, 'No':0, 'Yes':1}, inplace=True)
    values = data_raw[x].unique()
    print (x, sorted(values, key=lambda x: (str(type(x)), x)))  
    data[x] = data_raw[x]
    data_raw.drop([x], axis=1, inplace=True)

intersection [0, 1]
alcohol [0, 1]
roadway_departure [0, 1]
lane_departure [0, 1]
dr_sex_1 [0, 1]
dr_sex_2 [0, 1]


# Integer Values
- 'num_tot_inj'
- 'num_veh'
- 'parish_cd'
- 'parish_cd.1'

In [9]:
for x in ['num_tot_inj', 'num_veh', 'parish_cd', 'parish_cd.1']:
    data_raw[x].fillna(data_raw[x].mode(), inplace=True)
    values = data_raw[x].unique()
    print (x, sorted(values, key=lambda x: (str(type(x)), x)))  
    data[x] = data_raw[x]
    data_raw.drop([x], axis=1, inplace=True)    

num_tot_inj [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 36, 37, 52, 64]
num_veh [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 16]
parish_cd [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
parish_cd.1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]


# Values to Put in Ranges
- 'dr_age_1'
- 'dr_age_2'

In [10]:
for x in ['dr_age_1','dr_age_2']:
    data_raw[x].fillna(0, inplace=True)
    data_raw[x] = data_raw[x].astype(int)
    data_raw.loc[(data_raw[x]>100), x] = 0
    xbin = x + '_bin'
    data_raw[xbin] = data_raw[x].replace(0,np.nan)    
    data_raw[xbin] = pd.qcut(data_raw[xbin], 5, duplicates='drop').cat.add_categories(0)
    data_raw[xbin].fillna(0, inplace=True)    
    data[xbin] = data_raw[xbin]
    values = data_raw[xbin].unique()
    print (data[xbin].value_counts())
    data_raw.drop([x, xbin], axis=1, inplace=True)

(22.0, 30.0]     29092
(1.999, 22.0]    28982
(40.0, 56.0]     28270
(30.0, 40.0]     27890
(56.0, 99.0]     27462
0                18490
Name: dr_age_1_bin, dtype: int64
0                33099
(1.999, 26.0]    27664
(34.0, 45.0]     27379
(57.0, 97.0]     24573
(45.0, 57.0]     24222
(26.0, 34.0]     23249
Name: dr_age_2_bin, dtype: int64


# Distance from the Road
This one is weird.  

- The units are either in feet or miles.
- For many values the units are missing, and for others the measure is extreme. 
- I'm going to 
-- make all of the entries with missing units or negative distance zero, 
-- change all of the lengths to feet,
-- take out the zeroes, 
-- put in ranges,
-- and put the zeroes back.

In [11]:
for x in ['pri_measure']:
    data_raw[x] = data_raw[x].str.strip()
    values = data_raw[x].unique()
    print (x, len(values), values)
data_raw.loc[data_raw['pri_measure'] == '', 'pri_dist'] = 0
data_raw.loc[data_raw['pri_dist'] <= 0, 'pri_dist'] = 0
data_raw.loc[data_raw['pri_measure'] == 'MI', 'pri_dist'] *= 5280

for x in ['pri_dist']:
    data_raw[x].fillna(0, inplace=True)
#    data_raw[x] = data_raw[x].astype(int)
#    data_raw.loc[(data_raw[x]>100), x] = 0
    xbin = x + '_bin'
    data_raw[xbin] = data_raw[x].replace(0,np.nan)    
    data_raw[xbin] = pd.qcut(data_raw[xbin], 5, duplicates='drop').cat.add_categories(0)
    data_raw[xbin].fillna(0, inplace=True)    
    data[xbin] = data_raw[xbin]
    print (data_raw[xbin].value_counts())
    data_raw.drop([x, xbin], axis=1, inplace=True)
data_raw.drop(['pri_measure'], axis=1, inplace=True)

pri_measure 3 ['' 'FT' 'MI']
0                        60888
(-0.0009, 50.0]          25224
(150.0, 528.0]           22264
(2901.02, 52794720.0]    19860
(50.0, 150.0]            15993
(528.0, 2901.02]         15957
Name: pri_dist_bin, dtype: int64


# Alpha fields with 'Y' = 'Unknown' or 'Z' = 'Other'

We have lots of fields where 'Y' is 'Unknown' and 'Z' is 'Other.  
- Merge nan, blank, erroneous intergers, Y, and Z, into 'Z'. 

Other related Alpha fields:
- 'crash_type' does not have a Y or Z, and I can't figure out what it means.
- 'hwy_class' is mixed Alpha and integers, and I have no idea what it means.
- 'contributing_factor' has two values, 'R' and 'O', and I have no idea what it means.
- 'veh_severity' has five values, and I have no idea what it means.

These fields have trailing spaces I had to remove:
- 'f_harm_ev_cd1'
- 'm_harm_ev_cd1'

I lumped in some other fields here:
 - 'crash_type'
 - 'pri_contrib_fac_cd'
 - 'sec_pri_contrib_fac_cd'
 - 'hwy_type_cd'

In [12]:
for x in ['f_harm_ev_cd1', 'm_harm_ev_cd1']:
    data_raw[x] = data_raw[x].str.strip()

for x in data_raw:
    values = data_raw[x].unique()
    if (
        (('Y' in values or 'Z' in values) and len(values)<50)
        or x in ['crash_type', 'pri_contrib_fac_cd', 'sec_contrib_fac_cd', 'hwy_type_cd']
    ):
        print (x, sorted(values, key=lambda x: (str(type(x)), x)))
        data_raw[x].fillna('Z', inplace=True)
        data_raw[x].replace([' ', 'Y'], 'Z', inplace=True)
        data_raw[x] = data_raw[x].apply(lambda x: 'Z' if x.isnumeric() else x)
        values = data_raw[x].unique()
        print (x, sorted(values, key=lambda x: (str(type(x)), x)))
        print ()
        data[x] = data_raw[x]
        data_raw.drop([x], axis=1, inplace=True)
        
data_raw.drop(['hwy_class', 'contributing_factor'], axis=1, inplace=True)

f_harm_ev_cd1 ['', 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'HH', 'I', 'II', 'J', 'JJ', 'K', 'KK', 'L', 'LL', 'M', 'MM', 'N', 'NN', 'O', 'OO', 'P', 'PP', 'Q', 'QQ', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'YY', 'Z']
f_harm_ev_cd1 ['', 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'HH', 'I', 'II', 'J', 'JJ', 'K', 'KK', 'L', 'LL', 'M', 'MM', 'N', 'NN', 'O', 'OO', 'P', 'PP', 'Q', 'QQ', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'YY', 'Z']

m_harm_ev_cd1 ['', 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'HH', 'I', 'II', 'J', 'JJ', 'K', 'KK', 'L', 'LL', 'M', 'MM', 'N', 'NN', 'O', 'OO', 'P', 'PP', 'Q', 'QQ', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'YY', 'Z']
m_harm_ev_cd1 ['', 'A', 'AA', 'B', 'BB', 'C', 'CC', 'D', 'DD', 'E', 'EE', 'F', 'FF', 'G', 'GG', 'H', 'HH', 'I', 'II', 'J', 'JJ', 'K', 'KK', 'L', 'LL', 'M', 'MM', 'N', 'NN', 'O', 'OO', 'P', 'PP', 'Q', 'QQ', 'R', 'S', 'T', 'U', 'V', 'W

# Blank Field

In [13]:
for x in ['ped_actions_2']:
    data_raw.drop([x], axis=1, inplace=True)

# Remaining Fields
I don't know that any of these are likely to correlate.  

In [14]:
for x in data_raw:
    values = data_raw[x].unique()
    print (x, len(values))

route 1024
milepoint 53808
crash_num 160186
prior_movements 1322
csect 2141
logmile 18203
lrs_id 4788
lrs_logmile 18093
adt 749
intersection_id 15037
ORIG_LATITUDE 91540
ORIG_LONGITUDE 87921
DOTD_LATITUDE 127283
DOTD_LONGITUDE 130753
pri_hwy_num 1006
milepost 7172
pri_road_name 15724
inter_road 35371


# Fields in 'data' dataframe

In [15]:
for x in data:
    print (x)

fatal
crash_month
crash_dayofweek
crash_hour
pri_road_dir
intersection
alcohol
roadway_departure
lane_departure
dr_sex_1
dr_sex_2
num_tot_inj
num_veh
parish_cd
parish_cd.1
dr_age_1_bin
dr_age_2_bin
pri_dist_bin
f_harm_ev_cd1
m_harm_ev_cd1
man_coll_cd
crash_type
surf_cond_cd
invest_agency_cd
veh_type_cd1
veh_type_cd2
road_rel_cd
location_type
veh_severity_cd
hwy_type_cd
pri_contrib_fac_cd
sec_contrib_fac_cd
vision_obscure_1
vision_obscure_2
movement_reason_1
movement_reason_2
ped_actions_1
veh_lighting_1
veh_lighting_2
traff_cntl_cond_1
traff_cntl_cond_2
lighting_cd
dr_cond_cd1
dr_cond_cd2
veh_cond_cd1
veh_cond_cd2
